In [1]:
import keras
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import utilities as util
import pixel as pix
import tile

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1";

In [3]:
cnn_model = keras.models.load_model('/deep_data/resnet_1.h5')
cnn_small = keras.models.load_model('/deep_data/small_cnn.h5')

OSError: Unable to open file (unable to open file: name = '/deep_data/small_cnn.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 64
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen_cnn = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen_small = tile.tile_gen(landsat, sentinel, dem, label, 10, num_classes)
tile_gen_ml = tile.tile_gen(landsat, sentinel, dem, label, 1, num_classes)

In [ ]:
pixels = pixel_gen.gen_pixels(100000, not_include=5, balanced=True)

In [ ]:
train_px, val_px, test_px = pixel_gen.train_val_test_split(pixels, 0.7, 0.7)

In [ ]:
image_train = tile_gen_ml.tile_generator(train_px, len(train_px))
image_val = tile_gen_ml.tile_generator(val_px, len(val_px))
image_test = tile_gen_ml.tile_generator(test_px, len(test_px))

image_sk_train, label_sk_train = next(image_train) 
image_sk_val, label_sk_val = next(image_val) 
image_sk_test, label_sk_test = next(image_test)

image_sk_train = image_sk_train.reshape(len(train_px), 10)
image_sk_val = image_sk_val.reshape(len(val_px), 10)
image_sk_test = image_sk_test.reshape(len(test_px), 10)

In [ ]:
print(image_sk_train.shape)
print(label_sk_val.shape)
print(label_sk_test.shape)

In [ ]:
from sklearn import svm
import xgboost as xbg
import numpy as np

In [ ]:
clf = svm.SVC()

In [ ]:
clf.fit(image_sk_train, np.argmax(label_sk_train, axis=1))

In [ ]:
clf.score(image_sk_val, np.argmax(label_sk_val, axis=1)) 

In [ ]:
print(len(pred))

In [ ]:
batch_size = 25

In [ ]:
train_pred = clf.decision_function(image_sk_val)

In [ ]:
train_predictions_small = cnn_small.predict_generator(generator = tile_gen_small.tile_generator(val_px, batch_size=batch_size), steps=len(val_px) // batch_size, verbose=1)

In [ ]:
train_predictions = cnn_model.predict_generator(generator = tile_gen_cnn.tile_generator(val_px, batch_size=batch_size), steps=len(val_px) // batch_size, verbose=1)

In [ ]:
print(len(train_predictions))
input_data = list()
for i in range(len(train_predictions)):
    input_data.append(np.concatenate((train_predictions[i], train_predictions_small[i])))

In [ ]:
from keras import models
from keras import layers

In [ ]:
model = models.Sequential()
model.add(layers.Dense(128, activation = "relu", input_shape=(29,)))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(128, activation = "relu"))
model.add(layers.Dense(16, activation = "softmax"))

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(np.asarray(input_data), label_sk_val[:16275], epochs = 30, batch_size = 25)

In [ ]:
test_cnn = cnn_model.predict_generator(generator = tile_gen_cnn.tile_generator(test_px, batch_size=batch_size), steps=len(test_px) // batch_size, verbose=1)

In [ ]:
test_ml = clf.decision_function(image_sk_test)

In [ ]:
print(len(test_cnn))
print(len(test_ml))

In [ ]:
test_input_data = list()
for i in range(len(test_cnn)):
    test_input_data.append(np.concatenate((test_cnn[i], test_ml[i])))

In [ ]:
model.evaluate(x=np.asarray(test_input_data), y=label_sk_test[:len(test_input_data)], batch_size=128)